In [1]:
import torch
import numpy as np
import timm
import torchvision as tvision
import requests
import pandas as pd
import time

from PIL import Image
from urllib.request import urlopen
from transformers import AutoImageProcessor, ResNetForImageClassification

from art.estimators.classification import PyTorchClassifier
from art.attacks.evasion import (CarliniL2Method, ProjectedGradientDescent,
                                 ElasticNet, DeepFool, HopSkipJump)

class DoubleModelWrapper(torch.nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model.to(dtype=torch.double).cuda()

    def forward(self, x):
        x = x.to(dtype=torch.double)
        outputs = self.model(x)
        logits = outputs.logits.to(dtype=torch.double)
        return logits
    
class DoubleModelTimmWrapper(torch.nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model.to(dtype=torch.double)

    def forward(self, x):
        x = x.to(dtype=torch.double)
        outputs = self.model(x)
        return outputs.to(dtype=torch.double)


# Setting up models

mobile_model = timm.create_model('tf_mobilenetv3_large_minimal_100.in1k', pretrained=True)
mobile_model = mobile_model.cuda().eval()
mobile_model = DoubleModelTimmWrapper(mobile_model)

mobile_data_config = timm.data.resolve_model_data_config(mobile_model)
mobile_transforms = timm.data.create_transform(**mobile_data_config, is_training=False)

vgg16_model = timm.create_model('vgg16.tv_in1k', pretrained=True)
vgg16_model = vgg16_model.cuda().eval()
vgg16_model = DoubleModelTimmWrapper(vgg16_model)

vgg16_data_config = timm.data.resolve_model_data_config(vgg16_model)
vgg16_transforms = timm.data.create_transform(**vgg16_data_config, is_training=False)

processor = AutoImageProcessor.from_pretrained("microsoft/resnet-50")
hf_resnet = ResNetForImageClassification.from_pretrained("microsoft/resnet-50")
resnet_model = DoubleModelWrapper(hf_resnet)


# Downloading labels for ImageNet1k
url = "https://raw.githubusercontent.com/anishathalye/imagenet-simple-labels/master/imagenet-simple-labels.json"
imagenet_labels = requests.get(url).json()

# prepare image
# image = Image.open(urlopen("https://transforms.stlzoo.org/production/animals/red-kangaroo-02-01.jpg?w=1200&h=1200&auto=compress%2Cformat&fit=crop&dm=1654795233&s=5f137aa9a410a7ea3386c6972265111d"))

image_transforms = tvision.transforms.Compose([
    tvision.transforms.Resize((224, 224)),
    tvision.transforms.ToTensor()
])

# image = image_transforms(image).unsqueeze(0).numpy()

c:\Users\Wojciech\Desktop\ML-Advanced\Benchmark-of-Methods-for-Adversarial-Attacks\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Helpful Methods

In [2]:
def art_with_cw2(image_url: str, model : torch.nn.Module, target_id: int = 123):
    image = Image.open(urlopen(image_url))

    input_img = image_transforms(image).unsqueeze(0).numpy()

    classifier = PyTorchClassifier(
        model = model,
        loss = torch.nn.CrossEntropyLoss(),
        input_shape=(3,224,224),
        nb_classes=len(imagenet_labels),
        optimizer=None,
        clip_values=(0,1)
    )

    target_arr = np.zeros((1, len(imagenet_labels)))
    target_arr[0, target_id] = 1

    attack = CarliniL2Method(classifier=classifier, targeted=True, learning_rate=0.1, max_iter = 100, verbose=False)

    adv_img = attack.generate(x=input_img, y=target_arr)

    return torch.from_numpy(adv_img).cuda()

In [3]:
def art_with_pgd(image_url: str, model : torch.nn.Module, target_id: int = 123):
    image = Image.open(urlopen(image_url))

    input_img = image_transforms(image).unsqueeze(0).numpy()

    classifier = PyTorchClassifier(
        model = model,
        loss = torch.nn.CrossEntropyLoss(),
        input_shape=(3,224,224),
        nb_classes=len(imagenet_labels),
        optimizer=None,
        clip_values=(0,1)
    )

    target_arr = np.zeros((1, len(imagenet_labels)))
    target_arr[0, target_id] = 1

    attack = ProjectedGradientDescent(estimator=classifier, norm=np.inf, targeted=True, max_iter = 40, verbose=False)

    adv_img = attack.generate(x=input_img, y=target_arr)

    return torch.from_numpy(adv_img).cuda()

In [4]:
def art_with_deepfool(image_url: str, model : torch.nn.Module):
    image = Image.open(urlopen(image_url))

    input_img = image_transforms(image).unsqueeze(0).numpy()

    classifier = PyTorchClassifier(
        model = model,
        loss = torch.nn.CrossEntropyLoss(),
        input_shape=(3,224,224),
        nb_classes=len(imagenet_labels),
        optimizer=None,
        clip_values=(0,1)
    )

    attack = DeepFool(classifier=classifier, max_iter = 100, epsilon=0.000001, nb_grads=10, verbose=False)

    adv_img = attack.generate(x=input_img)

    return torch.from_numpy(adv_img).cuda()

In [5]:
def art_with_eaden(image_url: str, model : torch.nn.Module):
    image = Image.open(urlopen(image_url))

    input_img = image_transforms(image).unsqueeze(0).numpy()

    classifier = PyTorchClassifier(
        model = model,
        loss = torch.nn.CrossEntropyLoss(),
        input_shape=(3,224,224),
        nb_classes=len(imagenet_labels),
        optimizer=None,
        clip_values=(0,1)
    )

    attack = ElasticNet(classifier=classifier, max_iter = 10, decision_rule='EN', binary_search_steps=10, verbose=False)

    adv_img = attack.generate(x=input_img)

    return torch.from_numpy(adv_img).cuda()

In [ ]:
def art_with_hopskipjump(image_url: str, model : torch.nn.Module):
    image = Image.open(urlopen(image_url))

    input_img = image_transforms(image).unsqueeze(0).numpy()

    classifier = PyTorchClassifier(
        model = model,
        loss = torch.nn.CrossEntropyLoss(),
        input_shape=(3,224,224),
        nb_classes=len(imagenet_labels),
        optimizer=None,
        clip_values=(0,1)
    )

    attack = HopSkipJump(classifier=classifier, max_iter=10, verbose=False)

    adv_img = attack.generate(x=input_img)

    return torch.from_numpy(adv_img).cuda()

In [7]:
def benchmark_methods(methods: dict, image_url: str, model, num_of_iters: int = 5):
    """
    Benchmark a set of adversarial attack methods.

    Args:
        methods (dict): Dictionary of attack methods {method_name: function}.
        image_url (str): URL of the input image.
        model: Pretrained model to attack.

    Returns:
        pd.DataFrame: Benchmarking results as a tidy table.
    """
    results = []

    for method_name, method in methods.items():
        times = []
        probabilities_before = []
        probabilities_after = []
        label_before = []
        label_after = []
        # successes = []

        print(f"Benchmarking {method_name}...")

        for _ in range(num_of_iters):  # Run each method x times
            start_time = time.time()
            
            # Run the adversarial method
            raw_adversarial = method(image_url, model)
            
            img = Image.open(urlopen(image_url))
            input_tensor = image_transforms(img).unsqueeze(0).cuda()
            
            
            # Measure time
            elapsed_time = time.time() - start_time
            times.append(elapsed_time)
            
            # Collect probabilities
            with torch.no_grad():
                # For the original input
                logits_before = model(input_tensor)
                probabilities_before_run = torch.nn.functional.softmax(logits_before, dim=1)
                probabilities_before.append(torch.max(probabilities_before_run).cpu().detach().numpy())
                label_before.append(int(torch.argmax(logits_before).cpu().detach().numpy()))
                
                

                # For the adversarial input
                logits_after = model(raw_adversarial)
                probabilities_after_run = torch.nn.functional.softmax(logits_after, dim=1)
                probabilities_after.append(torch.max(probabilities_after_run).cpu().detach().numpy())
                label_after.append(int(torch.argmax(logits_after).cpu().detach().numpy()))
                
            
            # Track success
            # successes.append(success)
        
        # Store results for this method
        results.append({
            "Method": method_name,
            "Avg Time (s)": np.mean(times),
            "Avg Prob Before": np.mean(probabilities_before),
            "Avg Prob After": np.mean(probabilities_after),
            "Predicted Label Before": label_before,
            "Predicted Label After": label_after,
            # "Success Rate": np.mean(successes),
        })

    # Convert to DataFrame for a tidy table
    return pd.DataFrame(results)

In [8]:
mobile_classifier = PyTorchClassifier(
    model = mobile_model,
    loss = torch.nn.CrossEntropyLoss(),
    input_shape=(3,224, 224),
    nb_classes=len(imagenet_labels),
    optimizer=None,
    clip_values=(0,1)
)

vgg16_classifier = PyTorchClassifier(
    model = vgg16_model,
    loss = torch.nn.CrossEntropyLoss(),
    input_shape=(3,224,224),
    nb_classes=len(imagenet_labels),
    optimizer=None,
    clip_values=(0,1)
)

resnet_classifier = PyTorchClassifier(
    model = resnet_model,
    loss = torch.nn.CrossEntropyLoss(),
    input_shape=(3,224,224),
    nb_classes=len(imagenet_labels),
    optimizer=None,
    clip_values=(0,1)
)

In [9]:
methods = {
    'CarliniWagnerL2': art_with_cw2,
    'PGD': art_with_pgd,
    'DeepFool': art_with_deepfool,
    'EADEN': art_with_eaden,
    'HopSkipJump': art_with_hopskipjump,
}

image_url = 'https://as2.ftcdn.net/v2/jpg/03/05/26/83/1000_F_305268343_5Xi5esuvd6mIOqFC0QXZdCcqIWNQ6HR2.jpg'

## MobileNet

In [10]:
display(benchmark_methods(methods, image_url, mobile_model))

Benchmarking CarliniWagnerL2...
Benchmarking PGD...
Benchmarking DeepFool...
Benchmarking EADEN...
Benchmarking HopSkipJump...


KeyboardInterrupt: 

## Resnet50

In [ ]:
display(benchmark_methods(methods, image_url, resnet_model))

## VGG 16

In [ ]:
display(benchmark_methods(methods, image_url, vgg16_model))

Benchmarking PGD...


,Method,Avg Time (s),Avg Prob Before,Avg Prob After,Predicted Label Before,Predicted Label After
0,PGD,60.710116,0.185692,1.0,"[132, 132, 132, 132, 132, 132, 132]","[123, 123, 123, 123, 123, 123, 123]"
